<h2>심층신경망</h2>
<br>
<h3>2개의 층</h3>

입력층: 28X28크기의 784개 뉴런

은닉층: 출력층보다 개수가 큰 뉴런(활성화 함수)->비선형식을 만들어 선형식으로 합쳐지지 않도록 하기 위해 (시그모이드, 렐루)

출력층: 클래스 개수



심층신경망

은닉층 -> dense1

출력층 -> dense2

keras.Sequential([dense1,dense2])

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 # 정규화
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

심층 신경망

In [ ]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
#activation='sigmoid' : 시그모이드 활성화 함수
dense2 = keras.layers.Dense(10, activation='softmax')
#activation='softmax' : 소프트맥스 함수

심층 신경망 만들기

In [ ]:
model = keras.Sequential([dense1, dense2])
#순서: 은닉층, 출력층

In [ ]:
model.summary()
# 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력
#batch_size: 미니배치경사하강법, 기본값 32개, 크기변경가능,배치차원을 none으로 지정
#따라서 32개, 64개, 128개 batch_size 설정 가능
#params => 입력층(784)X은닉층(100)+100 = 가중치 + 절편 = 78500
#params => 은닉층(100)X출력층(10)+10 =가중치 + 절편 = 1010
# Non-trainable params: 경사 하강법으로 훈련되지 않는 파라미터 층의 파라미터 개수

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층을 추가하는 다른 방법2

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [ ]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층을 추가하는 다른 방법3

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5631 - accuracy: 0.8088
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4072 - accuracy: 0.8541
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3722 - accuracy: 0.8652
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3499 - accuracy: 0.8725
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3331 - accuracy: 0.8788


렐루 활성화 함수

렐루 함수와 Flatten 층

z>0이면 z값 출력

z<0이면 0으로 출력

시그모이드 함수의 단점 -> 선형출력 값이 넘 크거나 작은경우 시그모이드함수 변화가 작아짐 -> '포화가 되었다' -> 신경망 모델이 빠르게 학습하는 것을 할 수 없다 -> 층을 깊게 못함

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
#flatten층: 2차원 이미지를 1차원 배열로 바꾸어 준다 ->784개
#가중치 절편 모두 없기 때문에 Param=0  
#그래도 keras.layers밑에 있기 때문에 하나의 층으로 본다!
model.add(keras.layers.Dense(100, activation='relu'))#시그모이드 대신 렐루 함수사용
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
#확실히 렐루함수가 시그모이드 함수보다 성능이 쬐금 좋긴 함!

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5318 - accuracy: 0.8134
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3921 - accuracy: 0.8584
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3533 - accuracy: 0.8727
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3325 - accuracy: 0.8789
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3179 - accuracy: 0.8877


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3877 - accuracy: 0.8708


[0.38768866658210754, 0.8708333373069763]

옵티마이저 : 확률적 경사 하강법

최적화: SGD->모멘텀->네스테로브 모멘텀

모멘텀 + RMS prop = Adam -> 적응적 학습률 옵티마이저

확률적경사하강법: 천천히 내려가야함 learning rate-> 처음에는 경사가 급하다가 수렴점에는 조금씩 이동하는게 좋음 -> 즉 learning rate가 바뀌는 것이 좋다 = 적응적 학습률 옵티마이저



In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')
#문자열로 옵티마이저를 입력받는 경우 매개변수를 사용X, 기본값만

In [ ]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')
#sgd객체 전달->매개변수 사용 가능
#위의 코드랑 동일


In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.1)#기본값은 0.01

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
#모멘텀,네스토르브 모멘텀 매개변수 각각의 기본값 0,false

In [ ]:
# 적응적 학습률 사용하는 대표적인 옵티마이저 : Adagrad
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 적응적 학습률 사용하는 대표적인 옵티마이저 : RMSprop
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
#옵티마이저를 SGD,Adam,RMSprop, 등 여러개를 시도해보고 가장 잘 맞는 것을 찾는다.!
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5218 - accuracy: 0.8172
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3916 - accuracy: 0.8599
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3509 - accuracy: 0.8720
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3256 - accuracy: 0.8791
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3027 - accuracy: 0.8895


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 1ms/step - loss: 0.3381 - accuracy: 0.8767


[0.3380657136440277, 0.8766666650772095]

앙상블(랜덤포레스트)

1.여러개의 모델을 가지고 앙상블 함

2.각각의 개별 트리가 독립적으로 훈련후 합쳐짐

3.앙상블된 결과 위에 여러개의 층을 쌓을수 없다

4.그레이언트부스팅, 스태킹 은 순차적으로 훈련

5.DB, 엑셀

심층신경망

1.여러개의 유닛(은닉층)을 활용하여 출력층을 만든다.

2.은닉층의 유닛이 동시에 훈련

3.여러개의 층을 쌓을수 있다. 단계별 학습 가능

4.여러개의 층이 동시에 훈련

5.이미지, 텍스트